<a href="https://colab.research.google.com/github/highResearcher/Vicsek_interactive_simulation/blob/main/vicsek_github.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
plt.rcParams["animation.html"] = "jshtml"
from ipywidgets import interact, IntSlider, FloatSlider



def simulate_vicsek(box_length,rho,v,eta,steps):
  #parameters
  box_length=10.0 #length of the box
  n_particles=int(rho*box_length*box_length)
  r_int=1.0 #neighborhood radius

  #initialisation of positions and angles
  x_arr=np.zeros((steps,n_particles))
  y_arr=np.zeros((steps,n_particles))
  theta_arr=np.zeros((steps,n_particles))

  x_arr[0]=np.random.rand((n_particles))*box_length
  y_arr[0]=np.random.rand((n_particles))*box_length
  theta_arr[0]=np.random.rand((n_particles))*2.0*np.pi

  #evolution
  for time in range(1,steps):
    cos_thetas=np.zeros(n_particles)
    sin_thetas=np.zeros(n_particles)
    counter=np.zeros(n_particles)

    #theta update
    for i in range(n_particles):
      for j in range(n_particles):
        x_sepn=x_arr[time-1,i]-x_arr[time-1,j]
        y_sepn=y_arr[time-1,i]-y_arr[time-1,j]

        #min image convention
        x_sepn -= box_length* np.round(x_sepn/ box_length)
        y_sepn -= box_length* np.round(y_sepn/ box_length)

        sepn_sq=(x_sepn)**2 + (y_sepn)**2 #distance between the two particles
        if (sepn_sq<= r_int**2):
          cos_thetas[i]+= np.cos(theta_arr[time-1,j])
          sin_thetas[i]+= np.sin(theta_arr[time-1,j])
          counter[i]+= 1

    theta_arr[time]=np.atan2(sin_thetas,cos_thetas)+ (np.random.rand(n_particles)-0.5)*eta

    #position update
    x_arr[time]=x_arr[time-1]+ v*np.cos(theta_arr[time])
    y_arr[time]=y_arr[time-1]+ v*np.sin(theta_arr[time])
    #pbc
    for i in range(n_particles):
      if x_arr[time,i]> box_length: x_arr[time,i]= x_arr[time,i]-box_length
      if x_arr[time,i]< 0.0: x_arr[time,i]= x_arr[time,i]+ box_length
      if y_arr[time,i]> box_length: y_arr[time,i]= y_arr[time,i]-box_length
      if y_arr[time,i]< 0.0: y_arr[time,i]= y_arr[time,i]+ box_length

  return x_arr,y_arr,theta_arr



def vicsek(box_length=10.0,rho=0.9,v=0.02,eta=0.3,steps=1000):
  x,y,theta= simulate_vicsek(box_length=box_length,rho=rho,v=v,eta=eta,steps=steps)  #animation using func animation
  #define a figure first
  fig, ax= plt.subplots()
  ax.set_xlim([0,box_length])
  ax.set_ylim([0,box_length])
  ax.axis('square')
  line = ax.quiver(x[0],y[0],np.cos(theta[0]),np.sin(theta[0]),width=0.003)

  #update frame function
  def update_frame(i):
    line.set_offsets(np.c_[x[i],y[i]])
    line.set_UVC(np.cos(theta[i]),np.sin(theta[i]))
    return line,

  #final animation
  anim= FuncAnimation(fig,update_frame,frames=np.arange(0,steps,10),interval=50,blit=True)
  plt.close(fig)
  # show the plot
  return anim


interact(
    vicsek,
    box_length=FloatSlider(min=5, max=20, step=1, value=10, description="Box size"),
    rho=FloatSlider(min=0.0,max=5.0,step=0.1,value=0.9,description='density'),
    v=FloatSlider(min=0.01, max=0.2, step=0.01, value=0.02, description="Speed"),
    eta=FloatSlider(min=0.0, max=6.28, step=0.1, value=0.1, description="Noise"),
    steps=IntSlider(min=50, max=5000, step=50, value=1000, description="Steps"),
)